# Notebook — Risk & Drawdown Report (S&D Scalping M5/M1)
Notebook ini membuat **risk report otomatis** dari file `trades_baseline.csv` (hasil notebook strategi).

Output utama:
- Equity curve (R cumulative)
- Max Drawdown (R) + durasi DD
- Losing / Winning streak (hist & distribusi)
- Worst month snapshot
- Monte Carlo resampling (DD distribution & quantiles)
- Tabel rekomendasi risk-per-trade untuk DD limit (mis. 10%)

> Semua berbasis **R-multiple**, sehingga independen dari ukuran lot/modal.


## 0) Setup & Load trades
Pastikan kamu sudah punya output dari notebook strategi:
- `out_snd_v0_1/trades_baseline.csv` (atau file trades lain)

Ubah `TRADES_CSV` jika path berbeda.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)

TRADES_CSV = "out_snd_v0_1/trades_baseline.csv"  # ganti bila perlu

trades = pd.read_csv(TRADES_CSV, parse_dates=["entry_time","exit_time"])
trades = trades.sort_values("entry_time").reset_index(drop=True)

assert "R" in trades.columns, "Kolom R tidak ditemukan. Pastikan trades CSV benar."
trades[["zone_id","side","entry_time","R","exit_reason"]].head(), trades.shape


## 1) Summary metrics (R-space)

In [ ]:
def profit_factor_from_r(r: pd.Series) -> float:
    r = r.astype(float)
    pos = r[r>0].sum()
    neg = r[r<0].sum()
    return (pos/abs(neg)) if neg < 0 else np.inf

def summary_metrics(trades: pd.DataFrame) -> dict:
    r = trades["R"].astype(float)
    return {
        "n": int(len(r)),
        "winrate": float((r>0).mean()),
        "avgR": float(r.mean()),
        "medianR": float(r.median()),
        "sumR": float(r.sum()),
        "profit_factor": float(profit_factor_from_r(r)),
        "tp_rate": float((trades["exit_reason"]=="TP").mean()) if "exit_reason" in trades.columns else np.nan,
        "sl_rate": float((trades["exit_reason"]=="SL").mean()) if "exit_reason" in trades.columns else np.nan,
        "time_rate": float((trades["exit_reason"]=="TIME").mean()) if "exit_reason" in trades.columns else np.nan,
    }

metrics = summary_metrics(trades)
metrics


## 2) Equity curve & Max Drawdown (R)

In [ ]:
r = trades["R"].astype(float)
equity = r.cumsum()
peak = equity.cummax()
dd = equity - peak

mdd_r = float(dd.min())
mdd_end = int(dd.idxmin())
mdd_peak = int(equity[:mdd_end+1].idxmax()) if mdd_end > 0 else 0

mdd_info = {
    "MDD_R": mdd_r,
    "peak_index": mdd_peak,
    "trough_index": mdd_end,
    "peak_time": str(trades.loc[mdd_peak,"entry_time"]),
    "trough_time": str(trades.loc[mdd_end,"entry_time"]),
    "drawdown_trades": int(mdd_end - mdd_peak),
}
mdd_info


In [ ]:
plt.figure()
plt.plot(trades["entry_time"], equity.values)
plt.title("Equity Curve (Cumulative R)")
plt.xlabel("Time")
plt.ylabel("Cumulative R")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(trades["entry_time"], dd.values)
plt.title("Drawdown (R)")
plt.xlabel("Time")
plt.ylabel("Drawdown R")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()


## 3) Drawdown duration (trade-based)

In [ ]:
def drawdown_durations(equity: pd.Series) -> pd.DataFrame:
    peak = equity.cummax()
    underwater = equity < peak
    durations = []
    i = 0
    n = len(equity)
    while i < n:
        if not underwater.iloc[i]:
            i += 1
            continue
        start = i
        peak_idx = int((equity.iloc[:start+1]).idxmax())
        while i < n and underwater.iloc[i]:
            i += 1
        end = i - 1
        recovery = i if i < n else None
        durations.append({
            "peak_idx": peak_idx,
            "start_idx": start,
            "end_idx": end,
            "recovery_idx": recovery,
            "duration_trades": int((recovery - peak_idx) if recovery is not None else (n - peak_idx)),
            "dd_min_R": float((equity.iloc[peak_idx:end+1] - equity.iloc[peak_idx]).min()),
        })
    return pd.DataFrame(durations)

dd_durs = drawdown_durations(equity)
dd_durs.sort_values("duration_trades", ascending=False).head(10)


## 4) Losing / Winning streaks

In [ ]:
def streak_stats(r: np.ndarray) -> dict:
    max_loss = max_win = 0
    cur_loss = cur_win = 0
    loss_runs = []
    win_runs = []
    for x in r:
        if x < 0:
            cur_loss += 1
            if cur_win > 0:
                win_runs.append(cur_win)
            cur_win = 0
            max_loss = max(max_loss, cur_loss)
        else:
            cur_win += 1
            if cur_loss > 0:
                loss_runs.append(cur_loss)
            cur_loss = 0
            max_win = max(max_win, cur_win)
    if cur_loss > 0:
        loss_runs.append(cur_loss)
    if cur_win > 0:
        win_runs.append(cur_win)
    return {
        "max_losing_streak": int(max_loss),
        "max_winning_streak": int(max_win),
        "loss_runs": loss_runs,
        "win_runs": win_runs,
    }

st = streak_stats(r.values)
{ k:v for k,v in st.items() if k not in ("loss_runs","win_runs") }


In [ ]:
loss_runs = pd.Series(st["loss_runs"], name="losing_streak_len")
win_runs  = pd.Series(st["win_runs"],  name="winning_streak_len")

loss_runs.describe(), win_runs.describe()


In [ ]:
plt.figure()
plt.hist(loss_runs.values, bins=range(1, int(loss_runs.max())+2))
plt.title("Histogram: Losing Streak Lengths")
plt.xlabel("Consecutive Losses")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

plt.figure()
plt.hist(win_runs.values, bins=range(1, int(win_runs.max())+2))
plt.title("Histogram: Winning Streak Lengths")
plt.xlabel("Consecutive Wins")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


## 5) Worst month snapshot

In [ ]:
t = trades.copy()
t["month"] = t["entry_time"].dt.to_period("M").astype(str)

monthly = t.groupby("month").apply(lambda g: pd.Series({
    "n": len(g),
    "winrate": (g["R"]>0).mean(),
    "pf": profit_factor_from_r(g["R"]),
    "avgR": g["R"].mean(),
    "sumR": g["R"].sum()
})).reset_index()

monthly.sort_values("sumR").head(12)


## 6) Monte Carlo: Distribusi Max Drawdown (R)

In [ ]:
def max_drawdown_r(r_seq: np.ndarray) -> float:
    eq = np.cumsum(r_seq)
    peak = np.maximum.accumulate(eq)
    dd = eq - peak
    return float(dd.min()) if len(dd) else 0.0

def monte_carlo_dd(r: np.ndarray, n_sims: int = 2000, seed: int = 42) -> pd.Series:
    rng = np.random.default_rng(seed)
    mdds = np.empty(n_sims, dtype=float)
    for i in range(n_sims):
        perm = rng.permutation(r)
        mdds[i] = max_drawdown_r(perm)
    return pd.Series(mdds, name="MDD_R")

mdds = monte_carlo_dd(r.values, n_sims=2000, seed=42)
mdds.describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])


In [ ]:
plt.figure()
plt.hist(mdds.values, bins=50)
plt.title("Monte Carlo Distribution: Max Drawdown (R)")
plt.xlabel("MDD (R)")
plt.ylabel("Count")
plt.tight_layout()
plt.show()


## 7) Tabel rekomendasi risk-per-trade untuk DD limit

In [ ]:
def max_risk_per_trade_for_dd(mdd_r: float, dd_limit_pct: float) -> float:
    if mdd_r >= 0:
        return np.nan
    return (dd_limit_pct / 100.0) / abs(mdd_r)

hist_mdd = mdd_r
mc_p95 = float(mdds.quantile(0.95))
mc_p99 = float(mdds.quantile(0.99))

limits = [5, 10, 12, 15, 20]
rows = []
for L in limits:
    rows.append({
        "dd_limit_%": L,
        "max_f_hist": max_risk_per_trade_for_dd(hist_mdd, L),
        "max_f_mc95": max_risk_per_trade_for_dd(mc_p95, L),
        "max_f_mc99": max_risk_per_trade_for_dd(mc_p99, L),
    })
risk_table = pd.DataFrame(rows)
risk_table


## 8) Export artifacts (csv/json/png)

In [ ]:
import os, json

OUT_DIR = "out_risk_report_v0_1"
os.makedirs(OUT_DIR, exist_ok=True)

monthly.to_csv(f"{OUT_DIR}/monthly.csv", index=False)
dd_durs.to_csv(f"{OUT_DIR}/dd_durations.csv", index=False)
mdds.to_csv(f"{OUT_DIR}/mdd_monte_carlo.csv", index=False)
risk_table.to_csv(f"{OUT_DIR}/risk_per_trade_table.csv", index=False)

summary = {
    "metrics": metrics,
    "mdd_info": mdd_info,
    "mc_mdd_percentiles": {
        "p50": float(mdds.quantile(0.50)),
        "p90": float(mdds.quantile(0.90)),
        "p95": float(mdds.quantile(0.95)),
        "p99": float(mdds.quantile(0.99)),
    }
}
with open(f"{OUT_DIR}/risk_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

# save key plots
plt.figure()
plt.plot(trades["entry_time"], equity.values)
plt.title("Equity Curve (Cumulative R)")
plt.xlabel("Time"); plt.ylabel("Cumulative R")
plt.xticks(rotation=30); plt.tight_layout()
plt.savefig(f"{OUT_DIR}/equity_curve.png", dpi=150)
plt.close()

plt.figure()
plt.plot(trades["entry_time"], dd.values)
plt.title("Drawdown (R)")
plt.xlabel("Time"); plt.ylabel("Drawdown R")
plt.xticks(rotation=30); plt.tight_layout()
plt.savefig(f"{OUT_DIR}/drawdown_curve.png", dpi=150)
plt.close()

plt.figure()
plt.hist(loss_runs.values, bins=range(1, int(loss_runs.max())+2))
plt.title("Histogram: Losing Streak Lengths")
plt.xlabel("Consecutive Losses"); plt.ylabel("Count")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/losing_streak_hist.png", dpi=150)
plt.close()

plt.figure()
plt.hist(mdds.values, bins=50)
plt.title("Monte Carlo Distribution: Max Drawdown (R)")
plt.xlabel("MDD (R)"); plt.ylabel("Count")
plt.tight_layout()
plt.savefig(f"{OUT_DIR}/mdd_monte_carlo_hist.png", dpi=150)
plt.close()

print("Saved report to:", OUT_DIR)
